In [2]:
from random import random
import os
import random
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import classification_report
from tqdm.auto import tqdm
import numpy as np
import torch
from torch.utils.data import Dataset
import re
from transformers import AutoConfig
from transformers import AutoTokenizer, BertForSequenceClassification
from torch.utils.tensorboard import SummaryWriter
import time

In [4]:
max_length = 512
batch_size = 4
learning_rate = 1e-5
epoch_num = 3
themes = {"动力", "价格", "内饰", "配置", "安全性", "外观", "操控", "油耗", "空间", "舒适性"}

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

checkpoint = "bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Using cuda device


In [5]:
def seed_everything(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(12)

In [6]:
class ChnSentiCorp(Dataset):
    def __init__(self, data_file):
        self.data = self.load_data(data_file)

    def load_data(self, data_file):
        themes = {"动力", "价格", "内饰", "配置", "安全性", "外观", "操控", "油耗", "空间", "舒适性"}
        Data = {}
        theme_sentiment_pattern = re.compile(r'(\S+?)#(-?\d+)')  
        with open(data_file, 'rt', encoding='utf-8') as f:
            for idx, line in enumerate(f):
                line = line.strip()

                matches = theme_sentiment_pattern.findall(line)

                if not matches:
                    raise ValueError(f"Line {idx + 1} does not contain any valid theme-label pair: {line}")

                comment = re.sub(theme_sentiment_pattern, "", line).strip()

                theme_sentiment_pairs = matches  

                total_sentiment = sum(int(sentiment) for _, sentiment in theme_sentiment_pairs)

                if total_sentiment > 0:
                    sentiment_label = 2  
                elif total_sentiment < 0:
                    sentiment_label = 0  
                else:
                    sentiment_label = 1  


                Data[idx] = {
                    'comment': comment.replace(" ", ""),  
                    'themes': [theme for theme, _ in theme_sentiment_pairs],  
                    'sentiment': sentiment_label  
                }

        return Data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

train_data = ChnSentiCorp('data/train.txt')
test_data = ChnSentiCorp('data/test.txt')

In [7]:
def collate_fn(batch_samples):
    batch_sentences, batch_labels = [], []

    for sample in batch_samples:
        batch_sentences.append(sample['comment'])
        batch_labels.append(int(sample['sentiment']))

    batch_inputs = tokenizer(
        batch_sentences,  
        max_length=max_length,  
        padding=True,  
        truncation=True,  
        return_tensors="pt",  
        return_attention_mask=True 
    )

    input_ids = batch_inputs['input_ids']
    attention_mask = batch_inputs['attention_mask']

    labels = torch.tensor(batch_labels, dtype=torch.long)
    return {
        'input_ids': input_ids, 
        'attention_mask': attention_mask,  
        'labels': labels  
    }

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [8]:
config = AutoConfig.from_pretrained(checkpoint)
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=3).to(device)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [9]:
total_train_step = 0
total_train_loss = 0.
best_f1_score = 0.
total_test_loss = 0

def train_loop(dataloader, model, optimizer, lr_scheduler, epoch, total_train_loss, total_train_step):
    progress_bar = tqdm(range(len(dataloader)),disable=True)
    progress_bar.set_description(f'loss: {0:>7f}')
    finish_step_num = epoch * len(dataloader)

    model.train()
    for step, batch_data in enumerate(dataloader, start=1):
        batch_data = {k: v.to(device) for k, v in batch_data.items()}
        outputs = model(**batch_data)
        loss, _ = outputs

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_train_loss += loss.item()
        total_train_step +=1
        progress_bar.set_description(f'loss: {total_train_loss / (finish_step_num + step):>7f}')
        progress_bar.update(1)
        if total_train_step % 100 == 0:
            print("训练次数:{}，loss:{}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)
    return total_train_loss, total_train_step

def test_loop(dataloader, model, epoch):
    true_labels, predictions = [], []
    model.eval()
    total_test_loss = 0
    with torch.no_grad():
        for step, batch_data in enumerate(dataloader, start=1):
            batch_data = {k: v.to(device) for k, v in batch_data.items()}
            outputs = model(**batch_data)
            loss, logits = outputs
            pred = logits.argmax(dim=-1)
            true_labels += batch_data['labels'].cpu().numpy().tolist()
            predictions += pred.cpu().numpy().tolist()
            total_test_loss += loss.item()

    print("整体测试集上的Loss:{}".format(total_test_loss))
    writer.add_scalar("test_loss", total_test_loss, epoch)

    metrics = classification_report(true_labels, predictions, output_dict=True)

    pos_p, pos_r, pos_f1 = metrics['2']['precision'], metrics['2']['recall'], metrics['2']['f1-score']  # 正向情感 (类别 2)

    neu_p, neu_r, neu_f1= metrics['1']['precision'], metrics['1']['recall'], metrics['1']['f1-score']   # 中性情感 (类别 1)

    neg_p, neg_r, neg_f1 = metrics['0']['precision'], metrics['0']['recall'], metrics['0']['f1-score'] # 负向情感 (类别 0)

    macro_f1 = metrics['macro avg']['f1-score']
    micro_f1 = metrics['weighted avg']['f1-score']
    accuracy = metrics['accuracy']
    writer.add_scalar("test_accuarcy", accuracy, epoch)

    print(f"Positive (2): Precision: {pos_p * 100:>0.2f} / Recall: {pos_r * 100:>0.2f} / F1: {pos_f1 * 100:>0.2f}")
    print(f"Neutral (1): Precision: {neu_p * 100:>0.2f} / Recall: {neu_r * 100:>0.2f} / F1: {neu_f1 * 100:>0.2f}")
    print(f"Negative (0): Precision: {neg_p * 100:>0.2f} / Recall: {neg_r * 100:>0.2f} / F1: {neg_f1 * 100:>0.2f}")
    print(f"Accuracy: {accuracy * 100:>0.2f}")
    print(f"Macro-F1: {macro_f1 * 100:>0.2f} / Micro-F1: {micro_f1 * 100:>0.2f}\n")

    return metrics


In [10]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num * len(train_dataloader),
)


In [11]:
writer = SummaryWriter(log_dir='themes_classification_logs' + '/' + time.strftime('%m-%d_%H.%M', time.localtime()))

for epoch in range(epoch_num):
    print(f"Epoch {epoch + 1}/{epoch_num}\n" + 30 * "-")
    total_train_loss, total_train_step= train_loop(train_dataloader, model, optimizer, lr_scheduler, epoch, total_train_loss, total_train_step)
    valid_scores = test_loop(test_dataloader, model, epoch)
    macro_f1, micro_f1 = valid_scores['macro avg']['f1-score'], valid_scores['weighted avg']['f1-score']
    f1_score = (macro_f1 + micro_f1) / 2
    if f1_score > best_f1_score:
        best_f1_score = f1_score
        print('saving new weights...\n')
        torch.save(
            model.state_dict(),
            f'epoch_{epoch + 1}_valid_macrof1_{(macro_f1 * 100):0.3f}_microf1_{(micro_f1 * 100):0.3f}_model_weights.bin'
        )

writer.close()
print("Done!")

Epoch 1/3
------------------------------
训练次数:100，loss:0.39815378189086914
训练次数:200，loss:0.7061346769332886
训练次数:300，loss:0.35079506039619446
训练次数:400，loss:0.7237738966941833
训练次数:500，loss:0.49723997712135315
训练次数:600，loss:0.479952335357666
训练次数:700，loss:0.6700111031532288
训练次数:800，loss:0.6364050507545471
训练次数:900，loss:0.9160412549972534
训练次数:1000，loss:0.4736132025718689
训练次数:1100，loss:0.1950397938489914
训练次数:1200，loss:0.8391523361206055
训练次数:1300，loss:1.1750240325927734
训练次数:1400，loss:1.1670982837677002
训练次数:1500，loss:0.5805022716522217
训练次数:1600，loss:0.8716732263565063
训练次数:1700，loss:0.5139517784118652
训练次数:1800，loss:0.8265998959541321
训练次数:1900，loss:0.41246238350868225
训练次数:2000，loss:0.46494239568710327
整体测试集上的Loss:438.3846437856555
Positive (2): Precision: 58.22 / Recall: 32.89 / F1: 42.03
Neutral (1): Precision: 74.54 / Recall: 92.54 / F1: 82.57
Negative (0): Precision: 60.11 / Recall: 23.57 / F1: 33.86
Accuracy: 72.26
Macro-F1: 52.82 / Micro-F1: 68.47

saving new weights...

Epoc